# Теория вероятностей и математическая статистика
## Распределение хи-квадрат ($\chi^2$). Доверительный интервал для дисперсии. Проверка гипотезы о независимости признаков
### Содержание

0. [Введение: распределение хи-квадрат, общая логика ДИ](#par0)
1. [Общий вид ДИ для дисперсии](#par1)
2. [Примеры аналитического решения задач](#par2)
3. [Примеры решения задач в Python](#par3)
4. [Проверка гипотезы о независимости категориальных признаков](#par4)

In [ ]:
# !pip install numpy matplotlib seaborn scipy -q

In [ ]:
# основные библиотеки
# для генерации данных
import numpy as np

# визуализация
import seaborn as sns
import matplotlib.pyplot as plt
# поменяем дефолтные параметры графиков
sns.set(style='darkgrid', palette='deep')
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['savefig.format'] = 'pdf'

# для красоты и высокого разрешения
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### 0. Введение: распределение хи-квадрат, общая логика ДИ <a name="par0"></a>

#### Распределение хи-квадрат (chi-squared)

Распределение хи-квадрат — одно из фундаментальных в статистике. Оно широко используется в статистическом выводе (оценке доверительных интервалов, в частности, для дисперсии) и для тестов хи-квадрат (например, при анализе таблиц сопряженности).

- **Определение**: распределение хи-квадрат — это непрерывное распределение, возникающее из суммы квадратов определенного количества независимых стандартных нормальных случайных величин: $ x = z_{1}^{2}+ \ldots + z_{k}^{2} \sim \chi ^{2}(k), \ z_{i}\sim N(0,1) $.
- **Предпосылки**:
    - Распределение хи-квадрат основано на нормальном распределении.
    - Исходные сл.в. должны быть независимыми
    - Ключевым параметром являются степени свободы (*df*), которые в простейшем случае равны количеству суммируемых переменных.
- **Свойства**:
    - Форма распределения хи-квадрат зависит от степеней свободы. Оно скошено вправо (длинный правый хвост) и становится более симметричным с увеличением степеней свободы. При большом числе степеней свободы распределение хи-квадрат аппроксимируется к нормальному.
    - Распределение определено только для неотрицательных значений.
    - *Математическое ожидание* распределения хи-квадрат равно числу его степеней свободы ($EX = df$), а *дисперсия* в два раза больше числа степеней свободы ($VarX = 2df$).

**Визуализируем** функцию плотности вероятности распределения хи-квадрат для разных степеней свободы с использованием Python. Построим графики плотности вероятности для $ df = 2, 4, 6, 8$ и $ 20 $, чтобы показать, как меняется его форма.

In [ ]:
from scipy.stats import chi2

# Задаем степени свободы
dfs = [2, 4, 6, 8, 20]

# Генерируем данные для визуализации
x = np.linspace(0, 20, 1000)
chi_square_distributions = [chi2.pdf(x, df) for df in dfs]

# Создаем графики
plt.figure(figsize=(10, 6))
for i, df in enumerate(dfs):
    plt.plot(x, chi_square_distributions[i], label=f'df = {df}')

plt.title('Распределение хи-квадрат для разного числа степеней свободы', fontsize=16)
plt.xlabel('Значение')
plt.ylabel('Плотность вероятности')
plt.legend()
plt.show()

Можем увидеть, что при небольшом числе степеней свободы распределение скошено вправо. С увеличением степеней свободы распределение становится более симметричным, а дисперсия возрастает.

Посчитаем $P(\chi^2_{1}\leq3)$. Для этого нам понадобится функция распределения: chi2.cdf В скобках указаваем интересующее значение сл.в., а также количество степеней свободы df

In [ ]:
chi2.cdf(3, df = 1)

Посчитаем $P(\chi^2_{200}\geq180)$. А затем сравним с результатом, полученным при помощи нормальной аппроксимации.

In [ ]:
chi2.sf(180, df = 200)

In [ ]:
from scipy.stats import norm
norm.sf(180, 200, 20)

Сравним также значение нижнего квартиля, рассчитанного по распределению хи-квадрат и в условиях нормальной аппроксимации. Сохраним df = 200.

In [ ]:
chi2.ppf(0.25, df = 200)

In [ ]:
norm.ppf(0.25, 200, 20)

#### Общая логика построения ДИ

Раньше мы уже изучили несколько видов доверительных интервалов, и в будущем нам также будут встречаться их вариации, поскольку ДИ, наравне с точечным оцениванием, является одним из основных методов проверки статистических гипотез.

Сформулируем логику построения ДИ в общем виде. Для построения любого ДИ нам понадобятся:
1. **Точечная оценка**, полученная из данных выборки в зависимости от параметра, для которого мы строим ДИ (например, среднее арифметическое или выборочная доля).
2. **Квантиль** распределения статистики критерия (критическое значение). Его уровень зависит от *уровня доверия*. При увеличении уровня доверия размер доверительного интервала также увеличивается.
3. **Стандартная ошибка (Standard Error / SE)** выборки.

Для пунктов 2 и 3 берут распределение, которое лучше всего описывает выборочное распределение точечной оценки параметра.

Общая формула для доверительного интервала:
$$ \text{CI} = \hat{\theta} \pm (\text{Quantile}) \times (\text{Standard Error}) $$

Часто $ (\text{Quantile}) \times (\text{Standard Error}) $ называют **предельной ошибкой выборки** (Margin of Error / ME). Тогда формулу можно сократить до:

$$ \text{CI} = \hat{\theta} \pm \text{ME} $$

### 1. Общий вид ДИ для дисперсии <a name="par1"></a>

Теперь опишем, как выглядит доверительный интервал для дисперсии.

При его построении используется распределение хи-квадрат, потому что выборочная дисперсия, правильно отмасштабированная, имеет распределение хи-квадрат в предположении, что данные взяты из нормального распределения.

Формула для $ (1-\alpha) \cdot 100\% $ доверительного интервала для дисперсии ($ \sigma^2 = Var(X)$) следующая:

$$
\dfrac{S_x^2 \cdot (n-1)}{\chi^2_{1 - \alpha/2, \ df=n-1}} \le Var(X) \le \dfrac{S_x^2 \cdot (n-1)}{\chi^2_{\alpha/2, \ df=n-1}}
$$,
где:
- $ n $ — размер выборки.
- $ S_x^2 = \widehat{Var(X)}$ — выборочная дисперсия; $ S_x^2 = \dfrac{\sum (x_i - \bar{x})^2}{n-1} $ (несмещенная оценка дисперсии).
- $ \chi^2_{1 - \alpha/2, \ df=n-1} $ и $ \chi^2_{\alpha/2, \ df=n-1} $ — критические значения хи-квадрат с $ n-1 $ степенями свободы.

Обратите внимание, что такой ДИ будет несимметричен, поскольку распределение хи-квадрат скошено, особенно при небольших размерах выборки.

Как можно увидеть из формулы $ S_x^2 $, формулу ДИ можно сократить до:
$$
\dfrac{\sum (x_i - \bar{x})^2}{\chi^2_{1 - \alpha/2, \ df=n-1}} \le Var(X) \le \dfrac{\sum (x_i - \bar{x})^2}{\chi^2_{\alpha/2, \ df=n-1}}
$$

**Интерпретация доверительного интервала для дисперсии** (в терминах многократного семплинга)

При уровне значимости $ \alpha = 0.05 $, получаем уровень доверия $ (1-\alpha) \cdot 100\% = 95\% = \beta $.

- **Интерпретация с сильным ограничением** (предполагается *постоянство стандартной ошибки*): если мы будем проводить аналогичные исследования на выборках одного и того же размера много раз, независимо друг от друга, то в 95% случаев истинное значение дисперсии будет лежать в пределах от [левая граница] до [правая граница] (в предположении о том, что предельная ошибка выборки/стандартная ошибка не изменяется от выборки к выборке).

- **Интерпретация с ослабленным ограничением** (более гибкая, допускает *изменчивость стандартной ошибки* между различными выборками): с 95%-ной уверенностью мы можем утверждать, что истинное значение дисперсии лежит в интервале от [левая граница] до [правая граница]. Если мы будем проводить аналогичное исследование на выборках одного и того же размера много раз, независимо друг от друга, 95% доверительных интервалов будут накрывать истинный параметр дисперсии

### 2. Примеры аналитического решения задач <a name="par2"></a>

#### Task 1: Анализ общественного мнения по экологической политике

Представим, что в государстве N проводится опрос общественного мнения о поддержке экологической политики, проводимой государством, для определения направления ее дальнейшего развития.

Правительство провело опрос, попросив респондентов оценить степень их поддержки по шкале от 1 до 10 (где 1 — решительное несогласие, а 10 — решительная поддержка). Результаты опроса случайной выборки из 20 человек следующие:
$$
X = \{ 6, 8, 3, 10, 10, 9, 9, 8, 6, 5, 3, 7, 7, 5, 7, 7, 7, 6, 7, 5 \}
$$

Предполагая, что выборка пришла к нам из сл. в., имеющей нормальное распределение, нам необходимо построить 95%-ый доверительный интервал для дисперсии.

Помня формулу ДИ для дисперсии, ограничимся расчетом суммы квадратичных отклонений элементов выборки от среднего арифметического.

В начале найдем $\bar{x}$:
$$
\bar{x} = \dfrac{6 + 8 + 3 + 10 + 10 + 9 + 9 + 8 + 6 + 5 + 3 + 7 + 7 + 5 + 7 + 7 + 7 + 6 + 7 + 5}{20} = 6.75
$$

Далее найдем $ \sum (x_i - \bar{x})^2 $:
$$
\sum_{i=1}^n (x_i - \bar{x})^2 = (6 - 6.75)^2 + ... + (5 - 6.75)^2 = 73.75
$$

Теперь найдем нужные квантили:
* $ \beta $ (уровень доверия) $ = 0.95  \Rightarrow \alpha $ (уровень значимости) $ = 1 - 0.95 = 0.05$  
* $ \chi^2_{1 - \alpha/2, \ df=n-1} = \chi^2_{0.975, \ df=19} \approx 32.852$
* $ \chi^2_{\alpha/2, \ df=n-1} = \chi^2_{0.025, \ df=19} \approx 8.907$

Теперь мы можем подставить все в формулу ДИ:
$$
\dfrac{\sum (x_i - \bar{x})^2}{\chi^2_{1 - \alpha/2, \ df=n-1}} \le Var(X) \le \dfrac{\sum (x_i - \bar{x})^2}{\chi^2_{\alpha/2, \ df=n-1}}
$$

$$
\dfrac{73.75}{32.852} \le Var(X) \le \dfrac{73.75}{8.907}
$$

$$
2.24 \le Var(X) \le 8.28
$$

Получили ДИ для дисперсии сл. в. $X$: $[2.24; 8.28]$.

Дополнительно сравним этот ДИ с выборочной оценкой дисперсии, для этого нам нужно разделить уже найденный числитель $ \sum_{i=1}^n (x_i - \bar{x})^2 $ на $ (n - 1) $:
$$
S_x^2 = \dfrac{73.75}{19} \approx 3.88
$$

Как видим, ДИ для дисперсии $[2.24; 8.28]$ **несимметричен** относительно оценки дисперсии $ S_x^2 = 3.88 $.

**Интерпретация**

Проинтерпретируем этот доверительный интервал: он означает, что, если мы будем проводить аналогичные исследования на выборках одного и того же размера ($ n = 20 $) много раз, независимо друг от друга, в 95% случаев истинное значение дисперсии будет лежать в пределах от $2.24$ до $8.28$ (в предположении о том, что предельная ошибка выборки/стандартная ошибка не изменяется от выборки к выборке).

#### Task 2: Анализ общественного мнения по реформе в здравоохранении

Негосударственная исследовательская организация провела социологическое исследование отношения к реформе в области здравоохранения, предложенной правительством государства N.

Размер выборки из сл. в. $X$ составил $150$ человек, поддержка реформы измерялась по шкале от 1 до 10 (где 1 — решительное несогласие, а 10 — решительная поддержка).

*Несмещенная оценка дисперсии* сл. в. $X$, рассчитанная по выборке, равна $3.15$. Постройте 99%-ый доверительный интервал для генерального параметра дисперсии $X$.
Рассчитайте необходимые квантили как из распределения хи-квадрат напрямую, так и используя аппроксимацию к нормальному распределению.

##### Решение без аппроксимации

Мы уже можем найти значение числителя для обеих границ ДИ:
$$
S_x^2 \cdot (n - 1)= 3.15 \cdot 149 = 469.35
$$

Осталось только найти значение нужных квантилей.

* $ \beta $ (уровень доверия) $ = 0.99  \Rightarrow \alpha $ (уровень значимости) $ = 1 - 0.99 = 0.01$  
* $ \chi^2_{1 - \alpha/2, \ df=n-1} = \chi^2_{0.995, \ df=149} \approx 197.211$
* $ \chi^2_{\alpha/2, \ df=n-1} = \chi^2_{0.005, \ df=149} \approx 108.291$

Тогда ДИ будет равен:
$$
\dfrac{S_x^2 \cdot (n-1)}{\chi^2_{1 - \alpha/2, \ df=n-1}} \le Var(X) \le \dfrac{S_x^2 \cdot (n-1)}{\chi^2_{\alpha/2, \ df=n-1}}
$$
$$
\dfrac{469.35}{197.211} \le Var(X) \le \dfrac{469.35}{108.291}
$$
$$
2.38 \le Var(X) \le 4.33
$$

Нашли 99%-й ДИ **без аппроксимации**: он равен $[2.38; 4.33]$

##### Решение с аппроксимацией к нормальному распределению

При аппроксимации изменится только значение квантилей, поскольку мы будем находить их с помощью нормального распределения со следующими параметрами:
$$
\chi^2_{df=149} \sim N(149, 149\cdot2)
$$
, поскольку у распределения хи-квадрат математическое ожидание равно числу степеней свободы, а дисперсия — числу степеней свободы, умноженному на 2.

Чтобы найти квантили из $N(149, 149\cdot2)$, мы в начале найдем квантили такого же уровня из стандартного нормального распределения, а затем с помощью операции, обратной к стандартизации, получим значения соответствующих квантилей для $N(149, 149\cdot2)$.

Верхний квантиль будет равен:
$$
\chi^2_{0.995, \ df=149} \approx N_{0.995} (149, 149\cdot2)
$$
$$
N_{0.995} (149, 149\cdot2) = 149 + \sqrt{149 \cdot 2} \cdot Z_{0.995} = 149 + \sqrt{149 \cdot 2} \cdot 2.58 \approx 193.538
$$

Нижний квантиль будет равен:
$$
\chi^2_{0.005, \ df=149} \approx N_{0.005} (149, 149\cdot2)
$$
$$
N_{0.005} (149, 149\cdot2) = 149 + \sqrt{149 \cdot 2} \cdot Z_{0.005} = 149 + \sqrt{149 \cdot 2} \cdot (-2.58) \approx 104.462
$$

Тогда 99%-й ДИ будет равен:
$$
\dfrac{S_x^2 \cdot (n-1)}{N_{0.995} (149, 149\cdot2)} \le Var(X) \le \dfrac{S_x^2 \cdot (n-1)}{N_{0.005} (149, 149\cdot2)}
$$
$$
\dfrac{469.35}{193.538} \le Var(X) \le \dfrac{469.35}{104.462}
$$
$$
2.43 \le Var(X) \le 4.49
$$

Нашли 99%-й ДИ **с аппроксимацией**: он равен $[2.43; 4.49]$

Таким образом,
* ДИ **без аппроксимации**: $[2.38; 4.33]$
* ДИ **с аппроксимацией**: $[2.43; 4.49]$

При этом, несмещенная оценка дисперсии была равна $3.15$. Можно заметить, что несимметричность нашего ДИ сократилась примерно в два раза по сравнению с предыдущей задачей, когда размер выборки был равен 20.

### 3. Примеры решения задач в Python <a name="par3"></a>

Решим задачи с помощью Python.

Для расчета квантилей будем использовать функцию `ppf` (*percent point function*), которая есть у каждого распределения в библиотеке `scipy`.

Также обратите внимание, что `np.array` (аналог встроенного в Python `list`) поддерживает векторные операции, т.е. мы можем сразу писать «как в формуле»:
```python
((survey_data - np.mean(survey_data)) ** 2).sum() / (len(survey_data) - 1)  # выборочная оценка дисперсии
```

#### Задание 1

In [ ]:
# Документация: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html

import numpy as np
from scipy.stats import chi2

# Выборка
survey_data = np.array([6, 8, 3, 10, 10, 9, 9, 8, 6, 5, 3, 7, 7, 5, 7, 7, 7, 6, 7, 5])

# Размер выборки и выборочная дисперсия
n = len(survey_data)
# Сумма квадратов отклонений от среднего
sq_sum = ((survey_data - np.mean(survey_data)) ** 2).sum()

# Уровень значимости
alpha = 0.05

# Квантили распределения хи-квадрат
chi2_lower = chi2.ppf(alpha / 2, df=n-1)
chi2_upper = chi2.ppf(1 - alpha / 2, df=n-1)

# Границы ДИ
ci_lower = sq_sum / chi2_upper
ci_upper = sq_sum / chi2_lower

print(f"ДИ для дисперсии для выборки {survey_data} равен: \
      \n[{ci_lower:.3f}; {ci_upper:.3f}]")
print(f"Выборочная оценка дисперсии равна: {np.var(survey_data, ddof=1):.3f}")  # ddof - сколько вычесть из n для нормирования

Получили идентичный результат!

#### Задание 2

Воспроизведем решение как без аппроксимации, так и с ней:

In [ ]:
# Документация: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

import numpy as np
from scipy.stats import norm

# Параметры выборки
n = 150
var_est = 3.15
alpha = 0.01

# Точные квантили распределения хи-квадрат
chi2_lower_eq = chi2.ppf(alpha / 2, df=n-1)
chi2_upper_eq = chi2.ppf(1 - alpha / 2, df=n-1)

# Точный доверительный интервал
ci_lower_eq = var_est * (n - 1) / chi2_upper_eq
ci_upper_eq = var_est * (n - 1) / chi2_lower_eq

# Параметры нормального распределения для аппроксимации
mean_normal_approx = n - 1
std_dev_normal_approx = np.sqrt(2 * mean_normal_approx)

# Квантили стандартного нормального распределения
z_score_lower = norm.ppf(alpha / 2, 0, 1)
z_score_upper = norm.ppf(1 - alpha / 2, 0, 1)

# Аппроксимированные квантили распределения хи-квадрат
chi2_lower_approx = mean_normal_approx + std_dev_normal_approx * z_score_lower
chi2_upper_approx = mean_normal_approx + std_dev_normal_approx * z_score_upper

# Доверительный интервал с аппроксимацией
ci_lower_approx = var_est * (n - 1) / chi2_upper_approx
ci_upper_approx = var_est * (n - 1) / chi2_lower_approx

print(f"ДИ без аппроксимации: [{ci_lower_eq:.3f}; {ci_upper_eq:.3f}]")
print(f"ДИ с аппроксимацией : [{ci_lower_approx:.3f}; {ci_upper_approx:.3f}]")

Получили идентичный ответ!

### 4. Проверка гипотезы о независимости категориальных признаков <a name="par4"></a>

Данные для проверки такой гипотезы представлены в формате таблицы сопряженности. Стоит разделять таблицу сопряженности и таблицу совместного распределения: если во второй (таблице совместного распределения) в ячейках таблицы указаны вероятности комбинации определенных категорий признаков, то в первой (таблице сопряженности) - ячейки содержат наблюдаемые частоты (количество объектов, обладающих комбинацией признаков)

Рассмотрим следующую задачу для иллюстрации принципа применения критерия хи-квадрат в контексте проверки гипотезы о независимости категориальных признаков:

представим, что у нас есть данные за последний месяц работы небольшого кафе. Известно, что в кафе предлагают комплексный обед. Посетителю нужно выбрать одно блюдо и один напиток. Выбор у нас небольшой, впрочем, как и само кафе: только две опции по блюдам и две опции по напиткам. Признак А - выбор блюда - имеет две категории: A1 - пирожки, A2 - стейк из говядины. Признак B - выбор напитка - имеет две категории: B1 - черный чай, B2 - красное вино. Для начала зададим саму таблицу сопряженности:     

In [ ]:
table = np.array([[100,60],[40,200]])
table

Для удобства и более понятной таблицы добавим еще названия категорий:

In [ ]:
import pandas as pd
df = pd.DataFrame(table, columns = ['Black tea', 'Red wine'],  index = ['Pies', 'Steak'])
df

Теперь проверим гипотезу о независимости признаков, рассчитав соответствующую статистику критерия.

In [ ]:
import statsmodels.api as sm

In [ ]:
table = sm.stats.Table(df)
print(table.test_nominal_association())

В выдаче мы видим последовательно df (количество степеней свободы), pvalue = 0, и наконец, наблюдаемое значение статистики. Судя по этим результатам, мы имеем основание отвергнуть нулевую гипотезу, а значит выбор блюда и напитка не являются независимыми.

Отдельно можно также вывести таблицу ожидаемых значений при условии независимости.

In [ ]:
table.fittedvalues

Остатки позволят нам определить, какие сочетания признаков мы можем наблюдать с большей вероятностью.

In [ ]:
table.resid_pearson

Остатки больше 3 по модулю. Положительный знак - у сочетаний "черный чай и пирожки", а также "вино и красное вино". Это говорит о том, что эти сочетания мы наблюдаем чаще, чем при условии независимости.